In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random

from copy import deepcopy
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
from tqdm.notebook import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

from src.augmentations.CustomAugmentations import CustomAugmentationsTF
import matplotlib.pyplot as plt
import pickle
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

import os 
os.environ["PYTHONHASHSEED"] = "42"

keras.utils.set_random_seed(42)


In [2]:
height,width=32,32
batch_size=64

In [3]:

# Define the augmentation probabilities
p_dict = {
  # "flip": 0.5,
  # "transpose": 0.5,
  "gauss_noise": 0.5,
  # "brightness_contrast": 0.5,
  # "hue_saturation_value": 0.5,
}

p_dicts = []
# for aug_type in p_dict.keys():
#     p_d = {aug_type: .5}
#     p_dicts.append(p_d)

# p_dict = {
#   "flip": 0.5,
#   # "transpose": 0.5,
#   "gauss_noise": 0.5,
#   # "brightness_contrast": 0.5,
#   # "hue_saturation_value": 0.5,
# }
# p_dicts.append(p_dict)

p_dicts


[{'gauss_noise': 0.5}]

In [4]:
EPOCHS = 13

histories = list()
accuracies = list()
class_accuracies = list()
conf_mats = list()

for augs in tqdm(p_dicts, desc="Augmentation"):
    aug_str = "_".join([key + str(value) for key, value in augs.items()])
    print(f"Augmentations: {aug_str}")

    for iteration in tqdm(range(5), desc="Iteration"):
        train_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/train",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=True,
            seed=iteration,
            validation_split=0.05,
            subset="training",
        )

        val_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/valid",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=True,
            seed=iteration,
            validation_split=0.05,
            subset="validation",
        )

        test_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/test",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=False,
        )

        dnn_model = Sequential()
        imported_model = tf.keras.applications.EfficientNetV2L(
            include_top=False,
            input_shape=(height, width, 3),
            pooling="max",
            classes=10,
            weights="imagenet",
        )

        dnn_model.add(imported_model)
        dnn_model.add(Flatten())
        dnn_model.add(Dense(2048, activation="relu"))
        dnn_model.add(Dense(1024, activation="relu"))
        dnn_model.add(Dense(10, activation="softmax"))

        dnn_model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"],
        )
       

        history = {
            "loss": list(),
            "accuracy": list(),
            "val_loss": list(),
            "val_accuracy": list(),
        }

        print(f"Fitting model {iteration}")
        for epoch in tqdm(range(EPOCHS), desc="Epoch"):
            print(f"Epoch {epoch + 1}/{EPOCHS}")
            custom_aug = CustomAugmentationsTF(augs)
            train_set_aug = train_set.map(lambda x, y: (custom_aug.augment(x), y))

            hist_ = dnn_model.fit(train_set_aug, validation_data=val_set, epochs=1)
            history["loss"].append(hist_.history["loss"])
            history["accuracy"].append(hist_.history["accuracy"])
            history["val_loss"].append(hist_.history["val_loss"])
            history["val_accuracy"].append(hist_.history["val_accuracy"])

        print(f"Predicting model {iteration}")
        preds = dnn_model.predict(test_set, verbose=2)
        preds = preds.argmax(axis=1)
        classes = test_set.class_names
        test_labels = list()
        for images, labels in test_set:
            class_labels = [int(label) for label in labels]
            test_labels.extend(class_labels)
        test_labels = np.array(test_labels)

        conf_mat = confusion_matrix(test_labels, preds)
        accuracy = accuracy_score(test_labels, preds)
        print(f"Accuracy: {accuracy} for model {iteration}")
        class_accuracy = conf_mat.diagonal() / conf_mat.sum(axis=1)

        print(f"Saving model {iteration}")
        dnn_model.save(f"../models/EfficientNet_13epochs_aug_{aug_str}_{iteration}.h5")
        histories.append(deepcopy(history))
        accuracies.append(deepcopy(accuracy))
        class_accuracies.append(deepcopy(class_accuracy))
        conf_mats.append(deepcopy(conf_mat))


    with open(f"results/HISTORY_EfficientNet_13epochs_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(histories, f)

    with open(f"results/ACCURACY_EfficientNet_13epochs_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(accuracies, f)

    with open(f"results/CLASS_ACCURACY_EfficientNet_13epochs_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(class_accuracies, f)

    with open(f"results/CONF_MAT_EfficientNet_13epochs_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(conf_mats, f)

Augmentation:   0%|          | 0/1 [00:00<?, ?it/s]

Augmentations: gauss_noise0.5


Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Found 90000 files belonging to 10 classes.
Using 85500 files for training.
Found 90000 files belonging to 10 classes.
Using 4500 files for validation.
Found 90000 files belonging to 10 classes.
Fitting model 0


Epoch:   0%|          | 0/13 [00:00<?, ?it/s]

...

Epoch 1/13
1336/1336 [==============================] - 228s 148ms/step - loss: 1.2345 - accuracy: 0.5646 - val_loss: 0.8943 - val_accuracy: 0.6782
Epoch 2/13
1336/1336 [==============================] - 194s 145ms/step - loss: 0.8739 - accuracy: 0.7002 - val_loss: 0.8615 - val_accuracy: 0.6956
Epoch 3/13
1336/1336 [==============================] - 193s 145ms/step - loss: 0.7224 - accuracy: 0.7541 - val_loss: 0.8262 - val_accuracy: 0.7278
Epoch 4/13
1336/1336 [==============================] - 193s 145ms/step - loss: 0.7157 - accuracy: 0.7559 - val_loss: 0.8010 - val_accuracy: 0.7280
Epoch 5/13
1336/1336 [==============================] - 192s 144ms/step - loss: 0.5850 - accuracy: 0.8037 - val_loss: 0.8319 - val_accuracy: 0.7307
Epoch 6/13
1336/1336 [==============================] - 193s 144ms/step - loss: 0.6377 - accuracy: 0.7812 - val_loss: 0.8351 - val_accuracy: 0.7136
Epoch 7/13
1336/1336 [==============================] - 193s 144ms/step - loss: 0.6229 - accuracy: 0.7862 - val_

Epoch:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch 1/13
1336/1336 [==============================] - 229s 149ms/step - loss: 1.2760 - accuracy: 0.5457 - val_loss: 0.9979 - val_accuracy: 0.6564
Epoch 2/13
1336/1336 [==============================] - 195s 146ms/step - loss: 0.9189 - accuracy: 0.6854 - val_loss: 0.9637 - val_accuracy: 0.6640
Epoch 3/13
1336/1336 [==============================] - 195s 146ms/step - loss: 0.7490 - accuracy: 0.7441 - val_loss: 0.8435 - val_accuracy: 0.7067
Epoch 4/13
1336/1336 [==============================] - 196s 147ms/step - loss: 1.0410 - accuracy: 0.6380 - val_loss: 1.1556 - val_accuracy: 0.5929
Epoch 5/13
1336/1336 [==============================] - 196s 147ms/step - loss: 0.9357 - accuracy: 0.6719 - val_loss: 0.8294 - val_accuracy: 0.7089
Epoch 6/13
1336/1336 [==============================] - 196s 147ms/step - loss: 0.6424 - accuracy: 0.7775 - val_loss: 0.8093 - val_accuracy: 0.7284
Epoch 7/13
1336/1336 [==============================] - 196s 147ms/step - loss: 0.5300 - accuracy: 0.8192 - val_

Epoch:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch 1/13
1336/1336 [==============================] - 231s 151ms/step - loss: 1.3002 - accuracy: 0.5304 - val_loss: 1.0034 - val_accuracy: 0.6513
Epoch 2/13
1336/1336 [==============================] - 197s 147ms/step - loss: 1.1444 - accuracy: 0.5989 - val_loss: 1.0532 - val_accuracy: 0.6218
Epoch 3/13
1336/1336 [==============================] - 197s 147ms/step - loss: 1.0172 - accuracy: 0.6458 - val_loss: 1.0060 - val_accuracy: 0.6484
Epoch 4/13
1336/1336 [==============================] - 196s 147ms/step - loss: 0.9004 - accuracy: 0.6863 - val_loss: 0.9175 - val_accuracy: 0.6796
Epoch 5/13
1336/1336 [==============================] - 196s 147ms/step - loss: 0.9324 - accuracy: 0.6748 - val_loss: 0.9431 - val_accuracy: 0.6878
Epoch 6/13
1336/1336 [==============================] - 196s 147ms/step - loss: 0.7625 - accuracy: 0.7368 - val_loss: 0.8817 - val_accuracy: 0.7100
Epoch 7/13
1336/1336 [==============================] - 197s 147ms/step - loss: 0.6780 - accuracy: 0.7668 - val_